# Settings all imports

In [2]:
# apt-get handle the installation and removal of software on Debian, Ubuntu, and related Linux distributions
!apt-get update

# installing needed packages
!apt install chromium-chromedriver
!pip install selenium
!pip install beautifulsoup4

# Copy the chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
# This make sure that the selenium chromedriver is in the path
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [3]:
from google.colab import drive
#drive.mount("/content/drive")
drive.mount("/content/ShardDrives")

import tqdm
import requests
import urllib.request

import numpy as np
import pandas as pd

from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from bs4.element import Comment
# from google.colab import drive

Mounted at /content/ShardDrives


# Create a web driver based on operating system. Please change the value of the `OPERATING_SYSTEM` constant to run this notebook on your system

In [4]:
OPERATING_SYSTEM = "linux"

if OPERATING_SYSTEM == "windows":
    executable_path = "drivers/windows/chromedriver.exe"
elif OPERATING_SYSTEM == "mac":
    executable_path = "drivers/mac/chromedriver"
elif OPERATING_SYSTEM == "linux":
    executable_path = "chromedriver"

base_url = 'https://www.jobs.bg/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path=executable_path, options=chrome_options)


"""url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
r = driver.get(url)
html = driver.execute_script("return document.documentElement.outerHTML")
soup = BeautifulSoup(html, 'html.parser')"""

'url = \'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging\'\nr = driver.get(url)\nhtml = driver.execute_script("return document.documentElement.outerHTML")\nsoup = BeautifulSoup(html, \'html.parser\')'

# Extracting the category names and ids so we can perform filters based on category

In [5]:
def extract_categories_dataframe():
    """"""
    url = "https://www.jobs.bg/index.php"
    r = driver.get(url)
    categories_button = driver.find_element_by_id("categoriesChip")
    categories_button.click()
    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')

    categories_elem = soup.find('div',attrs={'id':'categoriesSelectSheet'})
    categories_elements = categories_elem.find_all("div", attrs={"class": "mdc-chip mdc-ripple-upgraded"})

    categories_info = []
    categories_info.append((56))

    for category_element in categories_elements:
        category_id = category_element['id'].split("_")[-1]
        category_id = int(category_id)
        categories_info.append(category_id)

    #categories_info = pd.DataFrame(categories_info, columns=["category_id"])

    return categories_info

In [35]:
cats_selected = [1,25]
cats_selected

[1, 25]

In [14]:
def get_position_id(text_info_elem):
    """
    """
    class_id_elem = 'card__title mdc-typography mdc-typography--headline6 text-overflow'
    id_elem = text_info_elem.find('a',attrs={'class':class_id_elem})['href']
    position_id = id_elem.lstrip('job/')
    
    return position_id

def extract_job_position_id_as_array(base_url, text_info_elem): # , job_company_elem):
    """
    """
    position_id = get_position_id(text_info_elem)

    current_job_info = [position_id]
    
    return current_job_info

def get_number_of_positions(text_info_elem):
    """
    get the total number of positions on the page
    INPUT: text_info_elem - beautifulsoup html webpage
    """
    class_id_elem = 'height:25px;width:220px;font-weight:500;padding-bottom:5px;'
    id_elem = text_info_elem.find('td',attrs={'style':class_id_elem}).text
    num_postings = int(id_elem.split('от',1)[1].strip())
        
    return num_postings

In [16]:
url = "https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&categories%5B0%5D=25&term=#paging"
              
r = driver.get(url)
html = driver.execute_script("return document.documentElement.outerHTML")
soup = BeautifulSoup(html, 'html.parser')

get_number_of_positions(soup)

105

In [26]:
int(round(get_number_of_positions(soup)/15,0))

7

In [28]:
for page in tqdm.tqdm(range(0, int(round(get_number_of_positions(soup)/15,0)))):
  try:
      # This is the base url looping through all categories and specified pages in order
      url = "https://www.jobs.bg/front_job_search.php?frompage={}&add_sh=1&categories%5B0%5D={}&term=#paging".format(page*15, 1)
    
      r = driver.get(url)
      html = driver.execute_script("return document.documentElement.outerHTML")
      soup = BeautifulSoup(html, 'html.parser')
      
      class_id_elem = 'card__title mdc-typography mdc-typography--headline6 text-overflow'
      id_elem = soup.find_all('a',attrs={'class':class_id_elem})
  except Exception as e:
      print(f"Page number: {page}, url: {url}")
      print(f"Unexpected error: {e.message}")

100%|██████████| 7/7 [00:14<00:00,  2.02s/it]


In [33]:
def extract_jobs(start_page):
    """"""
    position_category_list = []

    for category in tqdm.tqdm(cats_selected):
        url = "https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&categories%5B0%5D={}&term=#paging".format(category)
              
        r = driver.get(url)
        html = driver.execute_script("return document.documentElement.outerHTML")
        soup = BeautifulSoup(html, 'html.parser')

        endd_page = int(round(get_number_of_positions(soup)/15,0))
            
        for page in tqdm.tqdm(range(start_page, endd_page)):
    
            try:
                # This is the base url looping through all categories and specified pages in order
                url = "https://www.jobs.bg/front_job_search.php?frompage={}&add_sh=1&categories%5B0%5D={}&term=#paging".format(page*15, category)
              
                r = driver.get(url)
                html = driver.execute_script("return document.documentElement.outerHTML")
                soup = BeautifulSoup(html, 'html.parser')
                
                class_id_elem = 'card__title mdc-typography mdc-typography--headline6 text-overflow'
                id_elem = soup.find_all('a',attrs={'class':class_id_elem})
            except Exception as e:
                print(f"Page number: {page}, url: {url}")
                print(f"Unexpected error: {e.message}")

            for i in range(0,15,1):
                try:
                    position_id = id_elem[i]['href'].lstrip('job/')
                    position_id = (position_id, category)
                    position_category_list.append(position_id)

                except Exception as e:
                    print(f"Curent page: {page}, {category}, {position_id}")
                    print(f"Unexpected error: {e}")

    df_columns = ["position_id", "category"]

    position_categofy_df = pd.DataFrame(position_category_list, columns=df_columns)

    return position_categofy_df

In [36]:
jobs = extract_jobs(0)

 50%|█████     | 1/2 [00:12<00:12, 12.93s/it]

Curent page: 5, 1, ('5789919', 1)
Unexpected error: list index out of range
Curent page: 5, 1, ('5789919', 1)
Unexpected error: list index out of range
Curent page: 5, 1, ('5789919', 1)
Unexpected error: list index out of range
Curent page: 5, 1, ('5789919', 1)
Unexpected error: list index out of range
Curent page: 5, 1, ('5789919', 1)
Unexpected error: list index out of range
Curent page: 5, 1, ('5789919', 1)
Unexpected error: list index out of range



100%|██████████| 2/2 [00:25<00:00, 12.52s/it]


In [32]:
jobs

,position_id,category
0,5810012,25
1,5831082,25
2,5826489,25
3,5798567,25
4,5834237,25
...,...,...
184,5796654,1
185,5796653,1
186,5796652,1
187,5796635,1


In [ ]:
jobs.to_csv("/content/ShardDrives/MyDrive/jobsbg_scrapped/categories_20210610.csv", index=False)

In [ ]:
%ls /content/ShardDrives/MyDrive/Podcast

 Archive/   categories_20210609.csv  'episode 4_data'/  'For Bllopers 2_data'/
